In [1]:
import os
import random
from io import open
import unicodedata
import string
import re

import torch
import torchaudio
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from pathlib import Path
import kaldi_io
import sys
import gc
import json
import time
from data_4 import AudioDataLoader, AudioDataset, pad_list

%matplotlib inline

print_use = False

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [2]:
# train_json = "data.json"
# test_json = "data_test.json"
train_json="/home1/meichaoyang/workspace/git/Listen-Attend-Spell/egs/aishell2/dump/train/deltatrue/data.json"
test_json="/home1/meichaoyang/workspace/git/Listen-Attend-Spell/egs/aishell2/dump/test/deltatrue/data.json"
batch_size = 32
maxlen_in = 100000
maxlen_out = 30
num_workers = 4

## 加载数据

In [3]:
# with open(train_json, 'rb') as f:
#             data = json.load(f)['utts']

In [4]:
# sorted_data = sorted(data.items(), key=lambda data: int(
#             data[1]['input']['shape'][0]), reverse=True)

In [3]:
tr_dataset = AudioDataset(train_json, batch_size,
                              maxlen_in, maxlen_out)

tr_loader = AudioDataLoader(tr_dataset, batch_size=1, num_workers=num_workers)


In [4]:
te_dataset = AudioDataset(test_json, batch_size,
                              maxlen_in, maxlen_out)
te_loader = AudioDataLoader(te_dataset, batch_size=1, num_workers=num_workers)

In [5]:
char_list = []
char_list_path = "/home1/meichaoyang/workspace/git/Listen-Attend-Spell/egs/aishell2/data/lang_1char/train_chars.txt"
with open(char_list_path, "r") as f:
    for line in f:
        data = line.split()
        char_list.append(data[0])

# 模型搭建

In [6]:
MAX_LENGTH= 200
SOS_token = 0
EOS_token = 1
os.environ["CUDA_VISIBLE_DEVICES"]="3"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Attention

In [7]:
class DotProductAttention(nn.Module):
    r"""Dot product attention.
    Given a set of vector values, and a vector query, attention is a technique
    to compute a weighted sum of the values, dependent on the query.

    NOTE: Here we use the terminology in Stanford cs224n-2018-lecture11.
    """

    def __init__(self):
        super(DotProductAttention, self).__init__()
        # TODO: move this out of this class?
        # self.linear_out = nn.Linear(dim*2, dim)

    def forward(self, queries, values):
        """
        Args:
            queries: N x To x H
            values : N x Ti x H

        Returns:
            output: N x To x H
            attention_distribution: N x To x Ti
        """
        batch_size = queries.size(0)
        hidden_size = queries.size(2)
        input_lengths = values.size(1)
        # (N, To, H) * (N, H, Ti) -> (N, To, Ti)
        attention_scores = torch.bmm(queries, values.transpose(1, 2))
        attention_distribution = F.softmax(
            attention_scores.view(-1, input_lengths), dim=1).view(batch_size, -1, input_lengths)
        # (N, To, Ti) * (N, Ti, H) -> (N, To, H)
        attention_output = torch.bmm(attention_distribution, values)
        # # concat -> (N, To, 2*H)
        # concated = torch.cat((attention_output, queries), dim=2)
        # # TODO: Move this out of this class?
        # # output -> (N, To, H)
        # output = torch.tanh(self.linear_out(
        #     concated.view(-1, 2*hidden_size))).view(batch_size, -1, hidden_size)

        return attention_output, attention_distribution

### 金字塔BLSTM

In [8]:
class pyramidalBLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout=0.0, bidirectional=True):
        super(pyramidalBLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers,
                           batch_first=True,
                           dropout=dropout,
                           bidirectional=bidirectional)
        
    def forward(self, padded_input, input_lengths):
        
        total_length = padded_input.size(1)  # get the max sequence length
        packed_input = pack_padded_sequence(padded_input, input_lengths,
                                            batch_first=True)
        packed_output, hidden = self.rnn(packed_input)
        output, _ = pad_packed_sequence(packed_output,
                                        batch_first=True,
                                        total_length=total_length)
        return output, hidden
        

In [9]:
a = [[[1,2,3],[4,5,6]],[[7,8,9],[10,11,12]]]
a = torch.tensor(a)

In [10]:
a = torch.rand([32, 2236, 512])
a.shape

torch.Size([32, 2236, 512])

In [11]:
# a.reshape(a.shape[0],a.shape[1]//2,a.shape[2]*2)

## 模型构建

In [12]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.0):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.first = True

        self.pyramidalBLSTM = pyramidalBLSTM(input_size, hidden_size, 1, dropout=dropout)

    def forward(self, input, input_lengths):

        output, hidden = self.pyramidalBLSTM(input, input_lengths)

        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [13]:
class Decoder(nn.Module):
    """
    """

    def __init__(self, vocab_size, embedding_dim, sos_id, eos_id, hidden_size,
                 num_layers, bidirectional_encoder=True):
        super(Decoder, self).__init__()
        # Hyper parameters
        # embedding + output
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.sos_id = sos_id  # Start of Sentence
        self.eos_id = eos_id  # End of Sentence
        # rnn
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional_encoder = bidirectional_encoder  # useless now
        self.encoder_hidden_size = hidden_size  # must be equal now
        # Components
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.rnn = nn.ModuleList()
        self.rnn += [nn.LSTMCell(self.embedding_dim +
                                 self.encoder_hidden_size, self.hidden_size)]
        for l in range(1, self.num_layers):
            self.rnn += [nn.LSTMCell(self.hidden_size, self.hidden_size)]
        self.attention = DotProductAttention()
        self.mlp = nn.Sequential(
            nn.Linear(self.encoder_hidden_size + self.hidden_size,
                      self.hidden_size),
            nn.Tanh(),
            nn.Linear(self.hidden_size, self.vocab_size))

    def zero_state(self, encoder_padded_outputs, H=None):
        N = encoder_padded_outputs.size(0)
        H = self.hidden_size if H == None else H
        return encoder_padded_outputs.new_zeros(N, H)

    def forward(self, padded_input, encoder_padded_outputs):
        """
        Args:
            padded_input: N x To
            # encoder_hidden: (num_layers * num_directions) x N x H
            encoder_padded_outputs: N x Ti x H

        Returns:
        """
        # *********Get Input and Output
        # from espnet/Decoder.forward()
        # TODO: need to make more smart way
        ys = [y[y != IGNORE_ID] for y in padded_input]  # parse padded ys
        # prepare input and output word sequences with sos/eos IDs
        eos = ys[0].new([self.eos_id])
        sos = ys[0].new([self.sos_id])
        ys_in = [torch.cat([sos, y], dim=0) for y in ys]
        ys_out = [torch.cat([y, eos], dim=0) for y in ys]
        # padding for ys with -1
        # pys: utt x olen
        ys_in_pad = pad_list(ys_in, self.eos_id)
        ys_out_pad = pad_list(ys_out, IGNORE_ID)
        # print("ys_in_pad", ys_in_pad.size())
        assert ys_in_pad.size() == ys_out_pad.size()
        batch_size = ys_in_pad.size(0)
        output_length = ys_in_pad.size(1)
        # max_length = ys_in_pad.size(1) - 1  # TODO: should minus 1(sos)?

        # *********Init decoder rnn
        h_list = [self.zero_state(encoder_padded_outputs)]
        c_list = [self.zero_state(encoder_padded_outputs)]
        for l in range(1, self.num_layers):
            h_list.append(self.zero_state(encoder_padded_outputs))
            c_list.append(self.zero_state(encoder_padded_outputs))
        att_c = self.zero_state(encoder_padded_outputs,
                                H=encoder_padded_outputs.size(2))
        y_all = []

        # **********LAS: 1. decoder rnn 2. attention 3. concate and MLP
        embedded = self.embedding(ys_in_pad)
        for t in range(output_length):
            # step 1. decoder RNN: s_i = RNN(s_i−1,y_i−1,c_i−1)
            rnn_input = torch.cat((embedded[:, t, :], att_c), dim=1)
            h_list[0], c_list[0] = self.rnn[0](
                rnn_input, (h_list[0], c_list[0]))
            for l in range(1, self.num_layers):
                h_list[l], c_list[l] = self.rnn[l](
                    h_list[l-1], (h_list[l], c_list[l]))
            rnn_output = h_list[-1]  # below unsqueeze: (N x H) -> (N x 1 x H)
            # step 2. attention: c_i = AttentionContext(s_i,h)
            att_c, att_w = self.attention(rnn_output.unsqueeze(dim=1),
                                          encoder_padded_outputs)
            att_c = att_c.squeeze(dim=1)
            # step 3. concate s_i and c_i, and input to MLP
            mlp_input = torch.cat((rnn_output, att_c), dim=1)
            predicted_y_t = self.mlp(mlp_input)
            y_all.append(predicted_y_t)

        y_all = torch.stack(y_all, dim=1)  # N x To x C
        # **********Cross Entropy Loss
        # F.cross_entropy = NLL(log_softmax(input), target))
        y_all = y_all.view(batch_size * output_length, self.vocab_size)
        ce_loss = F.cross_entropy(y_all, ys_out_pad.view(-1),
                                  ignore_index=IGNORE_ID,
                                  reduction='mean')

        return ce_loss

       

    def recognize_beam(self, encoder_outputs, char_list, args):
        """Beam search, decode one utterence now.
        Args:
            encoder_outputs: T x H
            char_list: list of character
            args: args.beam

        Returns:
            nbest_hyps:
        """
        # search params
        beam = args.beam_size
        nbest = args.nbest
        if args.decode_max_len == 0:
            maxlen = encoder_outputs.size(0)
        else:
            maxlen = args.decode_max_len

        # *********Init decoder rnn
        h_list = [self.zero_state(encoder_outputs.unsqueeze(0))]
        c_list = [self.zero_state(encoder_outputs.unsqueeze(0))]
        for l in range(1, self.num_layers):
            h_list.append(self.zero_state(encoder_outputs.unsqueeze(0)))
            c_list.append(self.zero_state(encoder_outputs.unsqueeze(0)))
        att_c = self.zero_state(encoder_outputs.unsqueeze(0),
                                H=encoder_outputs.unsqueeze(0).size(2))
        # prepare sos
        y = self.sos_id
        vy = encoder_outputs.new_zeros(1).long()

        hyp = {'score': 0.0, 'yseq': [y], 'c_prev': c_list, 'h_prev': h_list,
               'a_prev': att_c}
        hyps = [hyp]
        ended_hyps = []

        for i in range(maxlen):
            hyps_best_kept = []
            for hyp in hyps:
                # vy.unsqueeze(1)
                vy[0] = hyp['yseq'][i]
                embedded = self.embedding(vy)
                # embedded.unsqueeze(0)
                # step 1. decoder RNN: s_i = RNN(s_i−1,y_i−1,c_i−1)
                rnn_input = torch.cat((embedded, hyp['a_prev']), dim=1)
                h_list[0], c_list[0] = self.rnn[0](
                    rnn_input, (hyp['h_prev'][0], hyp['c_prev'][0]))
                for l in range(1, self.num_layers):
                    h_list[l], c_list[l] = self.rnn[l](
                        h_list[l-1], (hyp['h_prev'][l], hyp['c_prev'][l]))
                rnn_output = h_list[-1]
                # step 2. attention: c_i = AttentionContext(s_i,h)
                # below unsqueeze: (N x H) -> (N x 1 x H)
                att_c, att_w = self.attention(rnn_output.unsqueeze(dim=1),
                                              encoder_outputs.unsqueeze(0))
                att_c = att_c.squeeze(dim=1)
                # step 3. concate s_i and c_i, and input to MLP
                mlp_input = torch.cat((rnn_output, att_c), dim=1)
                predicted_y_t = self.mlp(mlp_input)
                local_scores = F.log_softmax(predicted_y_t, dim=1)
                # topk scores
                local_best_scores, local_best_ids = torch.topk(
                    local_scores, beam, dim=1)

                for j in range(beam):
                    new_hyp = {}
                    new_hyp['h_prev'] = h_list[:]
                    new_hyp['c_prev'] = c_list[:]
                    new_hyp['a_prev'] = att_c[:]
                    new_hyp['score'] = hyp['score'] + local_best_scores[0, j]
                    new_hyp['yseq'] = [0] * (1 + len(hyp['yseq']))
                    new_hyp['yseq'][:len(hyp['yseq'])] = hyp['yseq']
                    new_hyp['yseq'][len(hyp['yseq'])] = int(
                        local_best_ids[0, j])
                    # will be (2 x beam) hyps at most
                    hyps_best_kept.append(new_hyp)

                hyps_best_kept = sorted(hyps_best_kept,
                                        key=lambda x: x['score'],
                                        reverse=True)[:beam]
            # end for hyp in hyps
            hyps = hyps_best_kept

            # add eos in the final loop to avoid that there are no ended hyps
            if i == maxlen - 1:
                for hyp in hyps:
                    hyp['yseq'].append(self.eos_id)

            # add ended hypothes to a final list, and removed them from current hypothes
            # (this will be a probmlem, number of hyps < beam)
            remained_hyps = []
            for hyp in hyps:
                if hyp['yseq'][-1] == self.eos_id:
                    # hyp['score'] += (i + 1) * penalty
                    ended_hyps.append(hyp)
                else:
                    remained_hyps.append(hyp)

            hyps = remained_hyps
            if len(hyps) > 0:
                print('remeined hypothes: ' + str(len(hyps)))
            else:
                print('no hypothesis. Finish decoding.')
                break

            for hyp in hyps:
                print('hypo: ' + ''.join([char_list[int(x)]
                                          for x in hyp['yseq'][1:]]))
        # end for i in range(maxlen)
        nbest_hyps = sorted(ended_hyps, key=lambda x: x['score'], reverse=True)[
            :min(len(ended_hyps), nbest)]
        return nbest_hyps

In [14]:
class Seq2Seq(nn.Module):
    """Sequence-to-Sequence architecture with configurable encoder and decoder.
    """

    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, padded_input, input_lengths, padded_target):
        """
        Args:
            padded_input: N x Ti x D
            padded_targets: N x To
        """
        encoder_padded_outputs, _ = self.encoder(padded_input , input_lengths)
        loss = self.decoder(padded_target, encoder_padded_outputs)
        return loss
    
    def recognize(self, input, input_lengths, char_list, args):
        """Sequence-to-Sequence beam search, decode one utterence now.
        Args:
            input: T x D
            char_list: list of characters
            args: args.beam

        Returns:
            nbest_hyps:
        """
        encoder_outputs, _ = self.encoder(input, input_lengths)
#         print("encoder_outputs", encoder_outputs.squeeze(1).shape)
        
        nbest_hyps = self.decoder.recognize_beam(encoder_outputs.squeeze(0), char_list, args)
        return nbest_hyps


In [43]:
def parse_hypothesis(hyp, char_list):
    """Function to parse hypothesis

    :param list hyp: recognition hypothesis
    :param list char_list: list of characters
    :return: recognition text strinig
    :return: recognition token strinig
    :return: recognition tokenid string
    """
    # remove sos and get results
    tokenid_as_list = list(map(int, hyp['yseq'][1:]))
    token_as_list = [char_list[idx] for idx in tokenid_as_list]
    score = float(hyp['score'])

    # convert to string
    tokenid = " ".join([str(idx) for idx in tokenid_as_list])
    token = " ".join(token_as_list)
    text = "".join(token_as_list).replace('<space>', ' ')
    return text, token, tokenid, score

def add_results_to_json(js, nbest_hyps, char_list):
    """Function to add N-best results to json

    :param dict js: groundtruth utterance dict
    :param list nbest_hyps: list of hypothesis
    :param list char_list: list of characters
    :return: N-best results added utterance dict
    """
    # copy old json info
    new_js = dict()
    new_js['utt2spk'] = js['utt2spk']
    new_js['output'] = []

    for n, hyp in enumerate(nbest_hyps, 1):
        # parse hypothesis
        rec_text, rec_token, rec_tokenid, score = parse_hypothesis(
            hyp, char_list)

        # copy ground-truth
        out_dic = dict(js['output'][0].items())


        # add recognition results
        out_dic['rec_text'] = rec_text
        out_dic['score'] = score

        # add to list of N-best result dicts
        new_js['output'].append(out_dic)

        # show 1-best result
        if n == 1:
            print('groundtruth: %s' % out_dic['text'])
            print('prediction : %s' % out_dic['rec_text'])

    return new_js

## 单步训练

In [15]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

## 训练迭代

In [16]:
def trainIters(model, epoch, optimizier, print_every=10, plot_every=10, learning_rate=0.01):
    log = open('train.log', 'w')
    start = time.time()
    n_iters = len(tr_dataset)
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
#     training_pairs = random.choices(a, k=n_iters)
    
    criterion = nn.NLLLoss()

    for e in range(epoch):
        for i, (data) in enumerate(tr_loader):
            padded_input, input_lengths, padded_target = data
            padded_input, input_lengths, padded_target = data
            padded_input = padded_input.cuda()
            input_lengths = input_lengths.cuda()
            padded_target = padded_target.cuda()
#             print("padded_input:",padded_input,"\npadded_target",padded_target)
            loss = model(padded_input, input_lengths, padded_target)
    #         print(loss) #.requires_grad
            print_loss_total += float(loss)
            plot_loss_total += float(loss)

            optimizier.zero_grad()
            loss.backward()

            optimizier.step()

            if (i+1) % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                txt = 'Epoch %d | Iter %d | %s (%d %d%%) %.4f' % (e+1, i+1, timeSince(start, (e *n_iters +i+1) / (n_iters*epoch)),
                                             (i+1), (e *n_iters +i+1) / (n_iters*epoch) * 100, print_loss_avg)
                print(txt)
                log.write(txt + "\n")
                log.flush()
            if i+1 % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0

    log.close()

In [17]:
ys=[i.split() for i in ['873 93 4882 995 1485 1748 3315 1685 374 3071 66 1976 3071 2081 153 894 1940 3124 4958 106 3103 3746 406 2044 2090 4815 4514 1940 123 1556 1536 2807 87 5098 602 3071 1002 4958', '48 38 47 1500 4463 581 1847 421 58 1714 3205 184 1500 4463 1017 2879 4381 2120 4381 3205 1880 1880 822 4381 2120 4381 3205 1880 2841 3279 1718', '161 1398 87 1764 428 1956 3479 2045 2547 4620 428 1174 2187 3506 1764 3479 5093 128 577 1398 3069 228 428', '4336 1349 4626 1366 1735 525 2080 127 525 113 1990 1735 4571 1826 607 2612 1133 224 1735 4571 3780 153 4858 1062 1371 4189 2800 1251 871 1293 3071 1017 4856 1251 2970 4858 2034', '60 613 1960 1672 163 4891 4536 5093 5093 96 4378 4471 4471 1940 65 3071 3331 3749', '3611 347 1691 2888 1027 1017 1206 203 1248 4359 127 3069 871 525 61 3576 341 1880 2841 3279 1718', '1243 3747 2095 1413 2028 3237 93 3456 2701 3071 521 4815 279 613 2101 4764 4246 527 3460 2028 3237 93 3526 2701', '267 3564 1977 3339 4373 4840 601 2500 421 135 596 4672 580 479 2982 1159 132 525 358 4668 3339 4373 4174 4544', '1399 71 1248 366 87 58 627 421 4179 4500 5059 3071 436 1084 3561 584 254 358 2766 262 4500 3569 4378 4218', '63 3069 113 525 4908 63 526 871 3069 359 525 113 2728 359 58', '4176 1946 4126 1755 537 4381 538 3412 1059 1049 58 127 627', '3686 1016 2638 4385 953 4786 1686 2081 1274 2374 3071 1216 1424 2972 2981 687 358 362 4209 1980 2972 2981', '4354 2879 225 225 5156 2205 871 1685 338 1159 225 268 637 894 85 153 3561 3338 64 358 1438 1717 4189 123 603 1064', '2296 913 511 1691 2155 36 47 38 35 49 30 3486 3496 94 2101 1691 2155 101 125 1880 2841 12 23', '279 1568 2000 1735 4198 3607 2353 135 4894 622 1227 5186 4480 2768 78 3884 1366 901 3182 3312 4620 1956 1826 2028 3237', '1721 3995 3898 2190 716 1380 447 522 4867 4496 445 87 2362 3414 935 4512 2120 2476 1349 4904 2390 4475 1014']]

In [18]:
nonzero_sorted_idx=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [19]:
k=[ys[i] for i in nonzero_sorted_idx]

In [28]:
char_list

['<unk>',
 '<sos>',
 '<eos>',
 "'",
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '㶧',
 '䶮',
 '一',
 '丁',
 '七',
 '万',
 '丈',
 '三',
 '上',
 '下',
 '不',
 '与',
 '丐',
 '丑',
 '专',
 '且',
 '丕',
 '世',
 '丘',
 '丙',
 '业',
 '丛',
 '东',
 '丝',
 '丞',
 '丢',
 '两',
 '严',
 '丧',
 '个',
 '丫',
 '中',
 '丰',
 '串',
 '临',
 '丸',
 '丹',
 '为',
 '主',
 '丽',
 '举',
 '乃',
 '久',
 '么',
 '义',
 '之',
 '乌',
 '乍',
 '乎',
 '乏',
 '乐',
 '乒',
 '乓',
 '乔',
 '乖',
 '乘',
 '乙',
 '九',
 '乞',
 '也',
 '习',
 '乡',
 '书',
 '买',
 '乱',
 '乳',
 '乾',
 '了',
 '予',
 '争',
 '事',
 '二',
 '于',
 '亏',
 '云',
 '互',
 '五',
 '井',
 '亘',
 '亚',
 '些',
 '亟',
 '亡',
 '亢',
 '交',
 '亥',
 '亦',
 '产',
 '亨',
 '亩',
 '享',
 '京',
 '亭',
 '亮',
 '亲',
 '亳',
 '亵',
 '人',
 '亿',
 '什',
 '仁',
 '仄',
 '仅',
 '仆',
 '仇',
 '今',
 '介',
 '仍',
 '从'

In [29]:
input_size = 240

hidden_size = 256
vocab_size = len(char_list)
embedding_dim = 512
sos_id = 1
eos_id = 2
learning_rate = 1e-3
momentum = 0
l2 = 1e-5

IGNORE_ID=-1

encoder = Encoder(input_size, hidden_size, dropout=0.0)
decoder = Decoder(vocab_size, embedding_dim, sos_id, eos_id, hidden_size*2,
                 num_layers=1, bidirectional_encoder=True)

model = Seq2Seq(encoder, decoder)
print(model)
model.cuda()

optimizier = torch.optim.Adam(model.parameters(),
                                     lr=learning_rate,
#                                      momentum=momentum,
                                     weight_decay=l2)
trainIters(model, 20,optimizier, print_every=100)

Seq2Seq(
  (encoder): Encoder(
    (pyramidalBLSTM): pyramidalBLSTM(
      (rnn): LSTM(240, 256, batch_first=True, bidirectional=True)
    )
  )
  (decoder): Decoder(
    (embedding): Embedding(5221, 512)
    (rnn): ModuleList(
      (0): LSTMCell(1024, 512)
    )
    (attention): DotProductAttention()
    (mlp): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
      (1): Tanh()
      (2): Linear(in_features=512, out_features=5221, bias=True)
    )
  )
)
Epoch 1 | Iter 100 | 0m 19s (- 1921m 14s) (100 0%) 6.9498
Epoch 1 | Iter 200 | 0m 35s (- 1735m 55s) (200 0%) 6.4405
Epoch 1 | Iter 300 | 0m 50s (- 1649m 7s) (300 0%) 5.9445
Epoch 1 | Iter 400 | 1m 5s (- 1596m 54s) (400 0%) 5.5277
Epoch 1 | Iter 500 | 1m 19s (- 1543m 55s) (500 0%) 5.2196
Epoch 1 | Iter 600 | 1m 32s (- 1496m 18s) (600 0%) 5.0425
Epoch 1 | Iter 700 | 1m 45s (- 1460m 23s) (700 0%) 4.8887
Epoch 1 | Iter 800 | 1m 57s (- 1429m 26s) (800 0%) 4.7706
Epoch 1 | Iter 900 | 2m 10s (- 1404m 31s) (900 0%) 

Epoch 1 | Iter 13900 | 22m 55s (- 939m 50s) (13900 2%) 0.6887
Epoch 1 | Iter 14000 | 23m 3s (- 938m 28s) (14000 2%) 0.6457
Epoch 1 | Iter 14100 | 23m 11s (- 936m 46s) (14100 2%) 0.8462
Epoch 1 | Iter 14200 | 23m 18s (- 935m 2s) (14200 2%) 0.6419
Epoch 1 | Iter 14300 | 23m 26s (- 933m 10s) (14300 2%) 0.6565
Epoch 1 | Iter 14400 | 23m 33s (- 931m 42s) (14400 2%) 0.7115
Epoch 1 | Iter 14500 | 23m 41s (- 930m 6s) (14500 2%) 0.6494
Epoch 1 | Iter 14600 | 23m 49s (- 928m 37s) (14600 2%) 0.6845
Epoch 1 | Iter 14700 | 23m 57s (- 927m 7s) (14700 2%) 0.7099
Epoch 1 | Iter 14800 | 24m 4s (- 925m 25s) (14800 2%) 0.6685
Epoch 1 | Iter 14900 | 24m 11s (- 923m 49s) (14900 2%) 0.7111
Epoch 1 | Iter 15000 | 24m 19s (- 922m 31s) (15000 2%) 0.6619
Epoch 1 | Iter 15100 | 24m 27s (- 921m 21s) (15100 2%) 0.7663
Epoch 1 | Iter 15200 | 24m 35s (- 920m 4s) (15200 2%) 0.7495
Epoch 1 | Iter 15300 | 24m 43s (- 918m 48s) (15300 2%) 0.6648
Epoch 1 | Iter 15400 | 24m 51s (- 917m 30s) (15400 2%) 0.6735
Epoch 1 | Iter

Epoch 1 | Iter 27200 | 37m 33s (- 768m 19s) (27200 4%) 0.4921
Epoch 1 | Iter 27300 | 37m 38s (- 767m 8s) (27300 4%) 0.9454
Epoch 1 | Iter 27400 | 37m 43s (- 765m 49s) (27400 4%) 0.5622
Epoch 1 | Iter 27500 | 37m 47s (- 764m 29s) (27500 4%) 0.5359
Epoch 1 | Iter 27600 | 37m 52s (- 763m 5s) (27600 4%) 0.4515
Epoch 1 | Iter 27700 | 37m 56s (- 761m 42s) (27700 4%) 0.5733
Epoch 1 | Iter 27800 | 38m 1s (- 760m 18s) (27800 4%) 0.5011
Epoch 1 | Iter 27900 | 38m 5s (- 758m 57s) (27900 4%) 0.5427
Epoch 1 | Iter 28000 | 38m 10s (- 757m 35s) (28000 4%) 0.5663
Epoch 1 | Iter 28100 | 38m 15s (- 756m 25s) (28100 4%) 0.9051
Epoch 1 | Iter 28200 | 38m 19s (- 755m 3s) (28200 4%) 0.5676
Epoch 1 | Iter 28300 | 38m 24s (- 753m 44s) (28300 4%) 0.4689
Epoch 1 | Iter 28400 | 38m 28s (- 752m 19s) (28400 4%) 0.5350
Epoch 1 | Iter 28500 | 38m 32s (- 750m 53s) (28500 4%) 0.5361
Epoch 1 | Iter 28600 | 38m 36s (- 749m 34s) (28600 4%) 0.7963
Epoch 1 | Iter 28700 | 38m 41s (- 748m 11s) (28700 4%) 0.6221
Epoch 1 | Ite

Epoch 2 | Iter 11800 | 57m 41s (- 763m 51s) (11800 7%) 0.5295
Epoch 2 | Iter 11900 | 57m 49s (- 763m 38s) (11900 7%) 0.4993
Epoch 2 | Iter 12000 | 57m 57s (- 763m 24s) (12000 7%) 0.5319
Epoch 2 | Iter 12100 | 58m 4s (- 763m 4s) (12100 7%) 0.5091
Epoch 2 | Iter 12200 | 58m 12s (- 762m 49s) (12200 7%) 0.6231
Epoch 2 | Iter 12300 | 58m 19s (- 762m 25s) (12300 7%) 0.4889
Epoch 2 | Iter 12400 | 58m 26s (- 761m 52s) (12400 7%) 0.5257
Epoch 2 | Iter 12500 | 58m 32s (- 761m 18s) (12500 7%) 0.5116
Epoch 2 | Iter 12600 | 58m 39s (- 760m 44s) (12600 7%) 0.5779
Epoch 2 | Iter 12700 | 58m 45s (- 760m 10s) (12700 7%) 0.5385
Epoch 2 | Iter 12800 | 58m 52s (- 759m 35s) (12800 7%) 0.5443
Epoch 2 | Iter 12900 | 58m 58s (- 759m 1s) (12900 7%) 0.5284
Epoch 2 | Iter 13000 | 59m 6s (- 758m 43s) (13000 7%) 0.5239
Epoch 2 | Iter 13100 | 59m 14s (- 758m 32s) (13100 7%) 0.6309
Epoch 2 | Iter 13200 | 59m 22s (- 758m 17s) (13200 7%) 0.5070
Epoch 2 | Iter 13300 | 59m 30s (- 758m 2s) (13300 7%) 0.5375
Epoch 2 | Ite

Epoch 2 | Iter 25100 | 71m 9s (- 693m 59s) (25100 9%) 0.7124
Epoch 2 | Iter 25200 | 71m 14s (- 693m 24s) (25200 9%) 0.4719
Epoch 2 | Iter 25300 | 71m 19s (- 692m 48s) (25300 9%) 0.5121
Epoch 2 | Iter 25400 | 71m 24s (- 692m 12s) (25400 9%) 0.3880
Epoch 2 | Iter 25500 | 71m 29s (- 691m 35s) (25500 9%) 0.4050
Epoch 2 | Iter 25600 | 71m 34s (- 690m 59s) (25600 9%) 0.4559
Epoch 2 | Iter 25700 | 71m 39s (- 690m 22s) (25700 9%) 0.3942
Epoch 2 | Iter 25800 | 71m 44s (- 689m 48s) (25800 9%) 0.5108
Epoch 2 | Iter 25900 | 71m 49s (- 689m 13s) (25900 9%) 0.4109
Epoch 2 | Iter 26000 | 71m 54s (- 688m 37s) (26000 9%) 0.4233
Epoch 2 | Iter 26100 | 71m 59s (- 688m 4s) (26100 9%) 0.4802
Epoch 2 | Iter 26200 | 72m 4s (- 687m 31s) (26200 9%) 0.5446
Epoch 2 | Iter 26300 | 72m 9s (- 687m 0s) (26300 9%) 0.7411
Epoch 2 | Iter 26400 | 72m 14s (- 686m 24s) (26400 9%) 0.4533
Epoch 2 | Iter 26500 | 72m 19s (- 685m 47s) (26500 9%) 0.3523
Epoch 2 | Iter 26600 | 72m 24s (- 685m 11s) (26600 9%) 0.5067
Epoch 2 | Ite

...

Epoch 18 | Iter 13800 | 665m 4s (- 96m 11s) (13800 87%) 0.3520
Epoch 18 | Iter 13900 | 665m 11s (- 96m 3s) (13900 87%) 0.3748
Epoch 18 | Iter 14000 | 665m 19s (- 95m 55s) (14000 87%) 0.3347
Epoch 18 | Iter 14100 | 665m 27s (- 95m 47s) (14100 87%) 0.4863
Epoch 18 | Iter 14200 | 665m 34s (- 95m 40s) (14200 87%) 0.3495
Epoch 18 | Iter 14300 | 665m 42s (- 95m 32s) (14300 87%) 0.3524
Epoch 18 | Iter 14400 | 665m 49s (- 95m 24s) (14400 87%) 0.3863
Epoch 18 | Iter 14500 | 665m 57s (- 95m 16s) (14500 87%) 0.3362
Epoch 18 | Iter 14600 | 666m 4s (- 95m 8s) (14600 87%) 0.3689
Epoch 18 | Iter 14700 | 666m 12s (- 95m 0s) (14700 87%) 0.3732
Epoch 18 | Iter 14800 | 666m 19s (- 94m 52s) (14800 87%) 0.3566
Epoch 18 | Iter 14900 | 666m 26s (- 94m 45s) (14900 87%) 0.3888
Epoch 18 | Iter 15000 | 666m 34s (- 94m 37s) (15000 87%) 0.3649
Epoch 18 | Iter 15100 | 666m 41s (- 94m 29s) (15100 87%) 0.4464
Epoch 18 | Iter 15200 | 666m 49s (- 94m 21s) (15200 87%) 0.4546
Epoch 18 | Iter 15300 | 666m 56s (- 94m 13s) 

Epoch 18 | Iter 26700 | 678m 22s (- 78m 57s) (26700 89%) 0.2997
Epoch 18 | Iter 26800 | 678m 27s (- 78m 49s) (26800 89%) 0.3379
Epoch 18 | Iter 26900 | 678m 31s (- 78m 41s) (26900 89%) 0.3663
Epoch 18 | Iter 27000 | 678m 36s (- 78m 33s) (27000 89%) 0.3341
Epoch 18 | Iter 27100 | 678m 40s (- 78m 24s) (27100 89%) 0.3811
Epoch 18 | Iter 27200 | 678m 45s (- 78m 16s) (27200 89%) 0.3297
Epoch 18 | Iter 27300 | 678m 50s (- 78m 8s) (27300 89%) 0.6522
Epoch 18 | Iter 27400 | 678m 54s (- 78m 0s) (27400 89%) 0.3629
Epoch 18 | Iter 27500 | 678m 59s (- 77m 52s) (27500 89%) 0.3519
Epoch 18 | Iter 27600 | 679m 3s (- 77m 44s) (27600 89%) 0.2881
Epoch 18 | Iter 27700 | 679m 7s (- 77m 36s) (27700 89%) 0.3926
Epoch 18 | Iter 27800 | 679m 12s (- 77m 27s) (27800 89%) 0.3406
Epoch 18 | Iter 27900 | 679m 16s (- 77m 19s) (27900 89%) 0.3529
Epoch 18 | Iter 28000 | 679m 20s (- 77m 11s) (28000 89%) 0.3846
Epoch 18 | Iter 28100 | 679m 25s (- 77m 3s) (28100 89%) 0.6528
Epoch 18 | Iter 28200 | 679m 29s (- 76m 55s) 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch 20 | Iter 12600 | 738m 47s (- 21m 36s) (12600 97%) 0.3918
Epoch 20 | Iter 12700 | 738m 55s (- 21m 28s) (12700 97%) 0.3765
Epoch 20 | Iter 12800 | 739m 3s (- 21m 20s) (12800 97%) 0.3784
Epoch 20 | Iter 12900 | 739m 11s (- 21m 12s) (12900 97%) 0.3759
Epoch 20 | Iter 13000 | 739m 19s (- 21m 5s) (13000 97%) 0.3703
Epoch 20 | Iter 13100 | 739m 27s (- 20m 57s) (13100 97%) 0.4574
Epoch 20 | Iter 13200 | 739m 35s (- 20m 49s) (13200 97%) 0.3521
Epoch 20 | Iter 13300 | 739m 43s (- 20m 41s) (13300 97%) 0.3753
Epoch 20 | Iter 13400 | 739m 50s (- 20m 33s) (13400 97%) 0.3558
Epoch 20 | Iter 13500 | 739m 56s (- 20m 26s) (13500 97%) 0.3773
Epoch 20 | Iter 13600 | 740m 4s (- 20m 18s) (13600 97%) 0.3795
Epoch 20 | Iter 13700 | 740m 12s (- 20m 10s) (13700 97%) 0.3552
Epoch 20 | Iter 13800 | 740m 20s (- 20m 2s) (13800 97%) 0.3500
Epoch 20 | Iter 13900 | 740m 28s (- 19m 54s) (13900 97%) 0.3768
Epoch 20 | Iter 14000 | 740m 36s (- 19m 46s) (14000 97%) 0.3482
Epoch 20 | Iter 14100 | 740m 44s (- 19m 39s)

Epoch 20 | Iter 25600 | 752m 53s (- 4m 39s) (25600 99%) 0.3656
Epoch 20 | Iter 25700 | 752m 58s (- 4m 31s) (25700 99%) 0.3074
Epoch 20 | Iter 25800 | 753m 3s (- 4m 23s) (25800 99%) 0.4001
Epoch 20 | Iter 25900 | 753m 8s (- 4m 15s) (25900 99%) 0.3105
Epoch 20 | Iter 26000 | 753m 13s (- 4m 8s) (26000 99%) 0.3239
Epoch 20 | Iter 26100 | 753m 18s (- 4m 0s) (26100 99%) 0.3747
Epoch 20 | Iter 26200 | 753m 23s (- 3m 52s) (26200 99%) 0.4171
Epoch 20 | Iter 26300 | 753m 28s (- 3m 44s) (26300 99%) 0.5887
Epoch 20 | Iter 26400 | 753m 33s (- 3m 36s) (26400 99%) 0.3493
Epoch 20 | Iter 26500 | 753m 38s (- 3m 29s) (26500 99%) 0.2743
Epoch 20 | Iter 26600 | 753m 42s (- 3m 21s) (26600 99%) 0.3887
Epoch 20 | Iter 26700 | 753m 47s (- 3m 13s) (26700 99%) 0.3046
Epoch 20 | Iter 26800 | 753m 52s (- 3m 5s) (26800 99%) 0.3248
Epoch 20 | Iter 26900 | 753m 56s (- 2m 57s) (26900 99%) 0.3640
Epoch 20 | Iter 27000 | 754m 1s (- 2m 50s) (27000 99%) 0.3388
Epoch 20 | Iter 27100 | 754m 6s (- 2m 42s) (27100 99%) 0.3915

In [54]:
input_size = 240

hidden_size = 256
vocab_size = len(char_list)
embedding_dim = 512
sos_id = 1
eos_id = 2
learning_rate = 1e-3
momentum = 0
l2 = 1e-5

IGNORE_ID=-1

encoder1 = Encoder(input_size, hidden_size, dropout=0.2)
decoder1 = Decoder(vocab_size, embedding_dim, sos_id, eos_id, hidden_size*2,
                 num_layers=1, bidirectional_encoder=True)

model1 = Seq2Seq(encoder1, decoder1)
print(model1)
model1.cuda()

optimizier = torch.optim.Adam(model1.parameters(),
                                     lr=learning_rate,
#                                      momentum=momentum,
                                     weight_decay=l2)
trainIters(model1, 20,optimizier, print_every=100)

/home1/meichaoyang/anaconda3/lib/python3.8/site-packages/torch/nn/modules/rnn.py:57: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Seq2Seq(
  (encoder): Encoder(
    (pyramidalBLSTM): pyramidalBLSTM(
      (lstm1): LSTM(240, 256, batch_first=True, dropout=0.2, bidirectional=True)
      (lstm2): LSTM(1024, 256, batch_first=True, dropout=0.2, bidirectional=True)
      (lstm3): LSTM(1024, 256, batch_first=True, dropout=0.2, bidirectional=True)
    )
  )
  (decoder): Decoder(
    (embedding): Embedding(5221, 512)
    (rnn): ModuleList(
      (0): LSTMCell(1024, 512)
    )
    (attention): DotProductAttention()
    (mlp): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
      (1): Tanh()
      (2): Linear(in_features=512, out_features=5221, bias=True)
    )
  )
)
Epoch 1 | Iter 100 | 0m 47s (- 4636m 27s) (100 0%) 6.9207
Epoch 1 | Iter 200 | 1m 27s (- 4261m 11s) (200 0%) 6.4932
Epoch 1 | Iter 300 | 2m 7s (- 4140m 3s) (300 0%) 6.0741
Epoch 1 | Iter 400 | 2m 44s (- 4009m 5s) (400 0%) 5.6419
Epoch 1 | Iter 500 | 3m 22s (- 3941m 30s) (500 0%) 5.3040
Epoch 1 | Iter 600 | 3m 58s (- 3867m 38s) (600 

Epoch 1 | Iter 13000 | 56m 57s (- 2500m 22s) (13000 2%) 0.5697
Epoch 1 | Iter 13100 | 57m 12s (- 2491m 55s) (13100 2%) 0.6842
Epoch 1 | Iter 13200 | 57m 30s (- 2485m 42s) (13200 2%) 0.5302
Epoch 1 | Iter 13300 | 57m 49s (- 2479m 47s) (13300 2%) 0.5734
Epoch 1 | Iter 13400 | 58m 6s (- 2473m 18s) (13400 2%) 0.5343
Epoch 1 | Iter 13500 | 58m 20s (- 2464m 34s) (13500 2%) 0.5627
Epoch 1 | Iter 13600 | 58m 31s (- 2453m 20s) (13600 2%) 0.5605
Epoch 1 | Iter 13700 | 58m 44s (- 2443m 58s) (13700 2%) 0.5309
Epoch 1 | Iter 13800 | 58m 59s (- 2436m 3s) (13800 2%) 0.5290
Epoch 1 | Iter 13900 | 59m 11s (- 2426m 24s) (13900 2%) 0.5566
Epoch 1 | Iter 14000 | 59m 22s (- 2416m 1s) (14000 2%) 0.5084
Epoch 1 | Iter 14100 | 59m 33s (- 2406m 24s) (14100 2%) 0.6763
Epoch 1 | Iter 14200 | 59m 44s (- 2396m 28s) (14200 2%) 0.4995
Epoch 1 | Iter 14300 | 59m 55s (- 2386m 19s) (14300 2%) 0.5105
Epoch 1 | Iter 14400 | 60m 5s (- 2375m 52s) (14400 2%) 0.5616
Epoch 1 | Iter 14500 | 60m 16s (- 2366m 6s) (14500 2%) 0.49

Epoch 1 | Iter 26100 | 90m 44s (- 1938m 45s) (26100 4%) 0.4562
Epoch 1 | Iter 26200 | 90m 58s (- 1935m 50s) (26200 4%) 0.5081
Epoch 1 | Iter 26300 | 91m 13s (- 1933m 38s) (26300 4%) 0.6758
Epoch 1 | Iter 26400 | 91m 27s (- 1930m 53s) (26400 4%) 0.4269
Epoch 1 | Iter 26500 | 91m 42s (- 1928m 33s) (26500 4%) 0.3293
Epoch 1 | Iter 26600 | 91m 57s (- 1926m 9s) (26600 4%) 0.4584
Epoch 1 | Iter 26700 | 92m 14s (- 1924m 31s) (26700 4%) 0.3599
Epoch 1 | Iter 26800 | 92m 31s (- 1922m 54s) (26800 4%) 0.3811
Epoch 1 | Iter 26900 | 92m 49s (- 1921m 35s) (26900 4%) 0.4278
Epoch 1 | Iter 27000 | 93m 6s (- 1919m 55s) (27000 4%) 0.3745
Epoch 1 | Iter 27100 | 93m 24s (- 1918m 40s) (27100 4%) 0.4376
Epoch 1 | Iter 27200 | 93m 38s (- 1915m 53s) (27200 4%) 0.3839
Epoch 1 | Iter 27300 | 93m 47s (- 1911m 39s) (27300 4%) 0.7611
Epoch 1 | Iter 27400 | 93m 55s (- 1907m 1s) (27400 4%) 0.4335
Epoch 1 | Iter 27500 | 94m 3s (- 1902m 23s) (27500 4%) 0.4121
Epoch 1 | Iter 27600 | 94m 11s (- 1897m 49s) (27600 4%) 0.3

Epoch 2 | Iter 10300 | 120m 18s (- 1658m 10s) (10300 6%) 0.4180
Epoch 2 | Iter 10400 | 120m 29s (- 1656m 15s) (10400 6%) 0.4280
Epoch 2 | Iter 10500 | 120m 41s (- 1654m 29s) (10500 6%) 0.4100
Epoch 2 | Iter 10600 | 120m 51s (- 1652m 21s) (10600 6%) 0.3992
Epoch 2 | Iter 10700 | 121m 2s (- 1650m 17s) (10700 6%) 0.3966
Epoch 2 | Iter 10800 | 121m 13s (- 1648m 27s) (10800 6%) 0.3803
Epoch 2 | Iter 10900 | 121m 25s (- 1646m 44s) (10900 6%) 0.3825
Epoch 2 | Iter 11000 | 121m 36s (- 1644m 56s) (11000 6%) 0.3767
Epoch 2 | Iter 11100 | 121m 48s (- 1643m 10s) (11100 6%) 0.3807
Epoch 2 | Iter 11200 | 121m 59s (- 1641m 14s) (11200 6%) 0.4050
Epoch 2 | Iter 11300 | 122m 10s (- 1639m 25s) (11300 6%) 0.4921
Epoch 2 | Iter 11400 | 122m 21s (- 1637m 31s) (11400 6%) 0.4034
Epoch 2 | Iter 11500 | 122m 33s (- 1635m 43s) (11500 6%) 0.3887
Epoch 2 | Iter 11600 | 122m 44s (- 1633m 54s) (11600 6%) 0.3971
Epoch 2 | Iter 11700 | 122m 55s (- 1632m 6s) (11700 7%) 0.4062
Epoch 2 | Iter 11800 | 123m 6s (- 1630m 17

...

Epoch 18 | Iter 23800 | 966m 47s (- 118m 33s) (23800 89%) 0.3136
Epoch 18 | Iter 23900 | 966m 55s (- 118m 21s) (23900 89%) 0.4662
Epoch 18 | Iter 24000 | 967m 3s (- 118m 9s) (24000 89%) 0.3092
Epoch 18 | Iter 24100 | 967m 11s (- 117m 58s) (24100 89%) 0.2456
Epoch 18 | Iter 24200 | 967m 20s (- 117m 46s) (24200 89%) 0.2844
Epoch 18 | Iter 24300 | 967m 27s (- 117m 35s) (24300 89%) 0.2477
Epoch 18 | Iter 24400 | 967m 35s (- 117m 23s) (24400 89%) 0.3243
Epoch 18 | Iter 24500 | 967m 43s (- 117m 12s) (24500 89%) 0.2837
Epoch 18 | Iter 24600 | 967m 51s (- 117m 0s) (24600 89%) 0.2407
Epoch 18 | Iter 24700 | 967m 58s (- 116m 49s) (24700 89%) 0.2890
Epoch 18 | Iter 24800 | 968m 6s (- 116m 37s) (24800 89%) 0.2085
Epoch 18 | Iter 24900 | 968m 14s (- 116m 25s) (24900 89%) 0.2978
Epoch 18 | Iter 25000 | 968m 22s (- 116m 14s) (25000 89%) 0.2931
Epoch 18 | Iter 25100 | 968m 30s (- 116m 2s) (25100 89%) 0.4450
Epoch 18 | Iter 25200 | 968m 37s (- 115m 51s) (25200 89%) 0.2909
Epoch 18 | Iter 25300 | 968m 4

Epoch 19 | Iter 7700 | 992m 2s (- 94m 18s) (7700 91%) 0.2938
Epoch 19 | Iter 7800 | 992m 14s (- 94m 7s) (7800 91%) 0.2891
Epoch 19 | Iter 7900 | 992m 26s (- 93m 56s) (7900 91%) 0.3014
Epoch 19 | Iter 8000 | 992m 37s (- 93m 44s) (8000 91%) 0.2940
Epoch 19 | Iter 8100 | 992m 49s (- 93m 33s) (8100 91%) 0.2954
Epoch 19 | Iter 8200 | 993m 1s (- 93m 22s) (8200 91%) 0.3572
Epoch 19 | Iter 8300 | 993m 13s (- 93m 11s) (8300 91%) 0.2990
Epoch 19 | Iter 8400 | 993m 24s (- 93m 0s) (8400 91%) 0.2939
Epoch 19 | Iter 8500 | 993m 36s (- 92m 49s) (8500 91%) 0.2836
Epoch 19 | Iter 8600 | 993m 47s (- 92m 38s) (8600 91%) 0.2768
Epoch 19 | Iter 8700 | 993m 59s (- 92m 27s) (8700 91%) 0.3008
Epoch 19 | Iter 8800 | 994m 10s (- 92m 15s) (8800 91%) 0.3002
Epoch 19 | Iter 8900 | 994m 22s (- 92m 4s) (8900 91%) 0.3276
Epoch 19 | Iter 9000 | 994m 33s (- 91m 53s) (9000 91%) 0.3036
Epoch 19 | Iter 9100 | 994m 44s (- 91m 42s) (9100 91%) 0.2992
Epoch 19 | Iter 9200 | 994m 56s (- 91m 31s) (9200 91%) 0.2941
Epoch 19 | It

Epoch 19 | Iter 20600 | 1014m 4s (- 70m 9s) (20600 93%) 0.3386
Epoch 19 | Iter 20700 | 1014m 13s (- 69m 58s) (20700 93%) 0.2905
Epoch 19 | Iter 20800 | 1014m 21s (- 69m 47s) (20800 93%) 0.2872
Epoch 19 | Iter 20900 | 1014m 29s (- 69m 35s) (20900 93%) 0.2563
Epoch 19 | Iter 21000 | 1014m 37s (- 69m 24s) (21000 93%) 0.2945
Epoch 19 | Iter 21100 | 1014m 45s (- 69m 12s) (21100 93%) 0.2668
Epoch 19 | Iter 21200 | 1014m 54s (- 69m 1s) (21200 93%) 0.3057
Epoch 19 | Iter 21300 | 1015m 2s (- 68m 50s) (21300 93%) 0.2394
Epoch 19 | Iter 21400 | 1015m 10s (- 68m 38s) (21400 93%) 0.3088
Epoch 19 | Iter 21500 | 1015m 18s (- 68m 27s) (21500 93%) 0.2559
Epoch 19 | Iter 21600 | 1015m 26s (- 68m 16s) (21600 93%) 0.3784
Epoch 19 | Iter 21700 | 1015m 34s (- 68m 4s) (21700 93%) 0.3677
Epoch 19 | Iter 21800 | 1015m 42s (- 67m 53s) (21800 93%) 0.2491
Epoch 19 | Iter 21900 | 1015m 50s (- 67m 42s) (21900 93%) 0.2845
Epoch 19 | Iter 22000 | 1015m 58s (- 67m 30s) (22000 93%) 0.2944
Epoch 19 | Iter 22100 | 1016m 

Epoch 20 | Iter 4400 | 1036m 9s (- 45m 56s) (4400 95%) 0.3497
Epoch 20 | Iter 4500 | 1036m 24s (- 45m 45s) (4500 95%) 0.3027
Epoch 20 | Iter 4600 | 1036m 39s (- 45m 35s) (4600 95%) 0.3137
Epoch 20 | Iter 4700 | 1036m 52s (- 45m 24s) (4700 95%) 0.3204
Epoch 20 | Iter 4800 | 1037m 7s (- 45m 13s) (4800 95%) 0.3117
Epoch 20 | Iter 4900 | 1037m 22s (- 45m 2s) (4900 95%) 0.3164
Epoch 20 | Iter 5000 | 1037m 37s (- 44m 51s) (5000 95%) 0.3032
Epoch 20 | Iter 5100 | 1037m 52s (- 44m 40s) (5100 95%) 0.2983
Epoch 20 | Iter 5200 | 1038m 6s (- 44m 29s) (5200 95%) 0.3099
Epoch 20 | Iter 5300 | 1038m 21s (- 44m 18s) (5300 95%) 0.3091
Epoch 20 | Iter 5400 | 1038m 35s (- 44m 7s) (5400 95%) 0.3003
Epoch 20 | Iter 5500 | 1038m 50s (- 43m 56s) (5500 95%) 0.2903
Epoch 20 | Iter 5600 | 1039m 4s (- 43m 45s) (5600 95%) 0.2968
Epoch 20 | Iter 5700 | 1039m 18s (- 43m 34s) (5700 95%) 0.2869
Epoch 20 | Iter 5800 | 1039m 33s (- 43m 23s) (5800 95%) 0.3377
Epoch 20 | Iter 5900 | 1039m 47s (- 43m 12s) (5900 96%) 0.297

Epoch 20 | Iter 17300 | 1062m 1s (- 22m 4s) (17300 97%) 0.3567
Epoch 20 | Iter 17400 | 1062m 11s (- 21m 53s) (17400 97%) 0.3197
Epoch 20 | Iter 17500 | 1062m 22s (- 21m 42s) (17500 97%) 0.2988
Epoch 20 | Iter 17600 | 1062m 32s (- 21m 31s) (17600 98%) 0.2805
Epoch 20 | Iter 17700 | 1062m 42s (- 21m 19s) (17700 98%) 0.2711
Epoch 20 | Iter 17800 | 1062m 52s (- 21m 8s) (17800 98%) 0.2915
Epoch 20 | Iter 17900 | 1063m 2s (- 20m 57s) (17900 98%) 0.3331
Epoch 20 | Iter 18000 | 1063m 12s (- 20m 46s) (18000 98%) 0.2550
Epoch 20 | Iter 18100 | 1063m 22s (- 20m 35s) (18100 98%) 0.2832
Epoch 20 | Iter 18200 | 1063m 32s (- 20m 24s) (18200 98%) 0.2954
Epoch 20 | Iter 18300 | 1063m 42s (- 20m 12s) (18300 98%) 0.3408
Epoch 20 | Iter 18400 | 1063m 51s (- 20m 1s) (18400 98%) 0.3737
Epoch 20 | Iter 18500 | 1064m 1s (- 19m 50s) (18500 98%) 0.2856
Epoch 20 | Iter 18600 | 1064m 11s (- 19m 39s) (18600 98%) 0.2728
Epoch 20 | Iter 18700 | 1064m 22s (- 19m 28s) (18700 98%) 0.2907
Epoch 20 | Iter 18800 | 1064m 3

In [30]:
class Args(object):
    def __init__(self, beam_size, nbest, decode_max_len):
        self.beam_size = beam_size
        self.nbest = nbest
        self.decode_max_len = decode_max_len

In [42]:
b = 200
n = 20
sample = te_dataset[b][n][1]
args = Args(30, 1, sample["output"][0]["shape"][0]+1)

input_tensor = torch.tensor(kaldi_io.read_mat(sample["input"][0]["feat"]))
a = input_tensor.shape[0]//4*4
input_tensor = input_tensor[0:(a if a < MAX_LENGTH*4 else MAX_LENGTH*4 ),:]

g = input_tensor.shape[0]
print(torch.tensor([g]))

input_tensor = input_tensor.unsqueeze(0)
print("input_tensor:",input_tensor.shape)
b = model.recognize(input_tensor.to(device),torch.tensor([g]), char_list, args)
print("正解：", sample["output"][0]["text"])

tensor([452])
input_tensor: torch.Size([1, 452, 240])
remeined hypothes: 29
hypo: 雨
hypo: 依
hypo: 狙
hypo: 已
hypo: 以
hypo: 遇
hypo: 一
hypo: 与
hypo: 于
hypo: 玉
hypo: 虞
hypo: E
hypo: 预
hypo: 伊
hypo: 御
hypo: 羽
hypo: 余
hypo: 渝
hypo: 许
hypo: 怡
hypo: 徐
hypo: 尤
hypo: 渔
hypo: G
hypo: 瑜
hypo: 乙
hypo: 芋
hypo: U
hypo: 育
remeined hypothes: 30
hypo: 狙击
hypo: 雨季
hypo: 以及
hypo: 已经
hypo: 依据
hypo: 预计
hypo: 虞姬
hypo: 遇到
hypo: 依滴
hypo: 伊迪
hypo: 玉季
hypo: ED
hypo: 雨滴
hypo: 一滴
hypo: 遇记
hypo: 一起
hypo: 与急
hypo: EG
hypo: 雨洁
hypo: 遇及
hypo: 雨起
hypo: 一直
hypo: 遇起
hypo: 于急
hypo: 与几
hypo: 羽洁
hypo: 尤其
hypo: 一击
hypo: 依及
hypo: 已起
remeined hypothes: 22
hypo: 已经一
hypo: 以及一
hypo: 预计一
hypo: 依据一
hypo: 雨季香
hypo: 伊迪香
hypo: 依据虾
hypo: EDC
hypo: 一起一
hypo: 依据香
hypo: 狙击虾
hypo: 狙击一
hypo: 与几十
hypo: 雨季轩
hypo: 依滴虾
hypo: 玉季香
hypo: 玉季虾
hypo: 已经销
hypo: 一滴一
hypo: 虞姬香
hypo: 雨季加
hypo: 依据肖
remeined hypothes: 29
hypo: 雨季香伴
hypo: 伊迪香伴
hypo: 以及一一
hypo: 已经一一
hypo: 依据香伴
hypo: 已经一十
hypo: 以及一些
hypo: 依据虾伴
hypo: 依据肖传
hypo: 已经一些
hypo: 以及一十
hypo: 预计一一
hypo

In [ ]:
with open("result.txt", "w") as f:
    out = sys.stdout
    log = open("print.txt","a")
    sys.stdout = log
    for i in range(len(te_dataset)):
        for j in range(len(te_dataset[i])):
            utt, sample = te_dataset[i][j][0],te_dataset[i][j][1]
            input_tensor = torch.tensor(kaldi_io.read_mat(sample["input"][0]["feat"]))
            a = input_tensor.shape[0]//4*4
            input_tensor = input_tensor[0:(a if a < MAX_LENGTH*4 else MAX_LENGTH*4 ),:]

            g = input_tensor.shape[0]
            print(torch.tensor([g]))

            input_tensor = input_tensor.unsqueeze(0)
            print("input_tensor:",input_tensor.shape)
            b = model.recognize(input_tensor.to(device),torch.tensor([g]), char_list, args)
            print("正解：", sample["output"][0]["text"])
            res = add_results_to_json(sample, b, char_list)
            f.write(utt+":\n"+"正解:"+res["output"][0]["text"]+"\n"+"预测:"+res["output"][0]["rec_text"]+"\n\n")
            f.flush()
            sys.stdout.flush()
    sys.stdout = out

In [41]:
len(tr_dataset)

29187

21

In [ ]:
for ch in data_list[1]["corpus"]:
    print(ch.item())

In [ ]:
d = torch.tensor([[[1,2,3],[4,5,6],[7,8,9],[10,11,12]]])

In [ ]:
lang.index2word[1]

In [ ]:
d.view((4,1,3))

In [ ]:
d[0]